# Biigle parser & frames extractor

This notebook is used to explore parsing the annotations from the Biigle reports & extract frames from the videos.

Check the streamlit page for a user friendly BIIGLE report parser.
https://spyfish-aotearoa.streamlit.app/Export_Biigle_Annotations


It allows you to export the reports, find MaxN, calculate sizes...
The notebook walks you through the process. Fill in the values in the next two cells, and run the remaining cells.


If there are any issues, or if it breaks please write an email to Kalindi, or open an issue on: 
https://github.com/wildlifeai/Spyfish-Aotearoa-toolkit/issues


In [ ]:
# DEV
# Uncomment, if you want to include local coding changes continuously.
%load_ext autoreload
%autoreload 2

# Biigle Info 

If you have your BIIGLE credentials set up as .nv variables, you can leave them here None.

Fill in the project & volume info

In [ ]:
# Your Biigle account
BIIGLE_API_EMAIL = None
# BIIGLE_API_EMAIL="kiwia@wildlife.ai"

# A token is a special password like number used for the Biigle API. 
# Find yours here: https://biigle.de/settings/tokens Keep this secret.
BIIGLE_API_TOKEN = None
# BIIGLE_API_TOKEN = "Mag1CN0"


# The ID of the video volume you want to export annotations from.
# You can find it on the url on biigle when you are on the page with all the clips:
# For example: https://biigle.de/volumes/25173, 25173 is the volume id.
# VOLUME_ID = "25173" # no sizes (yet.)
# VOLUME_ID = "25516" # has sizes
VOLUME_ID = "26577" 
PROJECT_ID = "3711"

In [ ]:
# Loading and installing the necessary libraries
try:
    from sftk.utils import ping
except ImportError:
    print("Downloading the Spyfish Aotearoa toolkit...")
    !pip install --upgrade --no-deps -q git+https://github.com/wildlifeai/Spyfish-Aotearoa-toolkit.git



from sftk.biigle_parser import BiigleParser

In [ ]:

biigle_parser = BiigleParser(email=BIIGLE_API_EMAIL, token=BIIGLE_API_TOKEN)

In [ ]:
processed_annotations = biigle_parser.process_video_annotations(VOLUME_ID, resource="volumes")
# Max counts every 30 seconds
max_n_30s_df = processed_annotations["max_n_30s_df"]
# Max count of whole video (used to determine where in the videos to annotate for size)
max_n_df = processed_annotations["max_n_df"]
# Potentially empty, if it was the video was not annotated for size.
sizes_df = processed_annotations["sizes_df"]

## Review the parsed dataframes

In [ ]:
max_n_30s_df.head()

In [ ]:
max_n_df.head()

In [ ]:

sizes_df.head()

In [ ]:
# Export the processed annotations into CVS files

max_n_30s_df.to_csv(f"{VOLUME_ID}_max_n_30s_df.csv", index=False)
max_n_df.to_csv(f"{VOLUME_ID}_max_n_df.csv", index=False)
sizes_df.to_csv(f"{VOLUME_ID}_sizes_df.csv", index=False)

In [ ]:
# Export for general annotations file
# # Not needed here, just added as a fyi
final_annotations_output_df = biigle_parser.format_count_annotations_output(max_n_df, interval_annotation_s=30)
final_annotations_output_df


## (WIP)Export annotated frames 

In [ ]:
print("rows:", len(max_n_30s_df))
print("columns:", list(max_n_30s_df.columns))
# display(max_n_30s_df.head(2))

In [ ]:

from sftk.wip.biigle_frames_extract import save_grabs_from_df
# you need to download the videos?
save_grabs_from_df(max_n_30s_df, "../data/biigle_files", "frames_out")


In [ ]:
import shutil, sys
print("python:", sys.executable)
print("ffmpeg:", shutil.which("ffmpeg"))
print("ffprobe:", shutil.which("ffprobe"))

## Extract frames from videos with annotations
TODO extract this box saving code into the sftk module after testing.

In [ ]:
# pip install opencv-python pandas numpy  # if you don't already have them

import ast, json, math, re
from pathlib import Path
from typing import Iterable, Optional
import cv2
import numpy as np
import pandas as pd

# --- helpers ---------------------------------------------------------------

def _tstamp_ms(t: float) -> str:
    """Format seconds -> HHMMSSmmm (matches your saved frame filenames)."""
    ms = int(round((t - math.floor(t)) * 1000))
    tot = int(math.floor(t))
    return f"{tot//3600:02d}{(tot//60)%60:02d}{tot%60:02d}{ms:03d}"

_clip_pat = re.compile(r"_clip_(\d+)_([0-9]+)\.", re.IGNORECASE)
def _is_clip(name: str) -> bool:
    return _clip_pat.search(name) is not None

def _parse_points(points_str: str) -> Optional[np.ndarray]:
    """
    BIIGLE 'points' may look like:
      "[[x1,y1,x2,y2,x3,y3,x4,y4]]"  OR  "[[[x1,y1],[x2,y2],[x3,y3],[x4,y4]]]"
    Returns Nx2 float array.
    """
    try:
        data = ast.literal_eval(points_str)
    except Exception:
        return None
    if isinstance(data, list) and data:
        inner = data[0]
        if isinstance(inner, list) and all(isinstance(v, (int, float)) for v in inner):
            return np.array(inner, dtype=float).reshape(-1, 2)
        if isinstance(inner, list) and all(isinstance(v, list) and len(v) == 2 for v in inner):
            return np.array(inner, dtype=float)
    return None

def _scale_pts_if_needed(pts: np.ndarray, img_w: int, img_h: int, attrs_json: str | None) -> np.ndarray:
    """If attributes has original width/height, scale polygon to actual image size."""
    if not attrs_json:
        return pts
    try:
        attrs = json.loads(attrs_json)
        w0, h0 = float(attrs.get("width", 0)), float(attrs.get("height", 0))
        if w0 > 0 and h0 > 0:
            return pts * np.array([img_w / w0, img_h / h0], dtype=float)
    except Exception:
        pass
    return pts

def _label_color_bgr(label: str) -> tuple[int,int,int]:
    """Deterministic pretty-ish color per label (BGR)."""
    h = abs(hash(label)) if label else 0
    return (50 + (h        % 206),
            50 + (h // 256 % 206),
            50 + (h // 65536 % 206))

def _find_frame(frames_dir: Path, video_filename: str, t_candidates: Iterable[float]) -> Optional[Path]:
    """
    Find an extracted frame matching <stem>__tHHMMSSmmm__*.jpg|png...
    Tries multiple time candidates (clip time, then original-time fallback).
    """
    stem = Path(video_filename).stem
    for t in t_candidates:
        tag = _tstamp_ms(float(t))
        for ext in ("jpg","png","jpeg","webp","bmp"):
            hits = list(frames_dir.glob(f"{stem}__t{tag}__*.{ext}"))
            if hits:
                # pick newest if multiple
                return sorted(hits, key=lambda p: p.stat().st_mtime, reverse=True)[0]
    return None

# --- main: overlay polygons onto existing frames ---------------------------

def overlay_boxes_on_frames(
    df: pd.DataFrame,
    frames_dir: str | Path,
    out_dir: str | Path,
    *,
    start_seconds_col: str = "start_seconds",
    line_thickness: int = 3,
    font_scale: float = 0.5,
    fill_alpha: float = 0.25,      # 0..1; set to 0 to disable fill
) -> dict:
    """
    Draw all df['points'] polygons onto the already-extracted images in frames_dir,
    writing <original>__boxed.jpg into out_dir. Returns {'drawn':N, 'skipped':M}.
    Required df columns: video_filename, points, frames. Optional: label_name, attributes, start_seconds.
    """
    need = {"video_filename","points","frames"}
    missing = need - set(df.columns)
    if missing:
        raise ValueError(f"DataFrame is missing required columns: {sorted(missing)}")

    frames_dir = Path(frames_dir).resolve()
    out_dir = Path(out_dir).resolve(); out_dir.mkdir(parents=True, exist_ok=True)

    # group all annotations by their target frame path so we load/write each image once
    groups: dict[Path, list[dict]] = {}

    for _, row in df.iterrows():
        vid = str(row["video_filename"])
        points_str = str(row["points"])
        label = str(row.get("label_name") or "")
        attrs = str(row.get("attributes") or "")

        # parse time(s): BIIGLE gives frames like "[16.02563]"
        try:
            frames_val = row["frames"]
            if isinstance(frames_val, str):
                frames_val = ast.literal_eval(frames_val)
            if isinstance(frames_val, list) and frames_val:
                t0 = float(frames_val[0])
            else:
                t0 = float(frames_val)
        except (ValueError, SyntaxError, TypeError, IndexError):
            continue

        # figure out which timestamp our saved frame used
        t_candidates = []
        if _is_clip(vid):
            t_candidates.append(t0)  # time within clip (how we saved frames in the fast/fixed code)
        if start_seconds_col in df.columns:
            try:
                t_candidates.append(float(row[start_seconds_col]) + t0)  # fallback to original time naming
            except Exception:
                pass
        if not _is_clip(vid):  # non-clip files
            t_candidates.append(t0)

        img_path = _find_frame(frames_dir, vid, t_candidates)
        if not img_path:
            continue

        pts = _parse_points(points_str)
        if pts is None or pts.size == 0:
            continue

        groups.setdefault(img_path, []).append({"pts": pts, "label": label, "attrs": attrs})

    drawn = skipped = 0
    for img_path, items in groups.items():
        img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        if img is None:
            skipped += 1
            continue
        h, w = img.shape[:2]
        canvas = img.copy()

        for it in items:
            pts = _scale_pts_if_needed(it["pts"], w, h, it["attrs"])
            pts_i = pts.astype(int).reshape(-1,1,2)
            color = _label_color_bgr(it["label"])

            if fill_alpha and 0 < fill_alpha <= 1:
                overlay = canvas.copy()
                cv2.fillPoly(overlay, [pts_i], color)
                cv2.addWeighted(overlay, fill_alpha, canvas, 1 - fill_alpha, 0, canvas)

            cv2.polylines(canvas, [pts_i], isClosed=True, color=color, thickness=line_thickness)

            if it["label"]:
                x, y = int(pts[0,0]), int(pts[0,1])
                cv2.putText(canvas, it["label"], (x, y-5),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                            fontScale=font_scale, color=color, thickness=2, lineType=cv2.LINE_AA)

        out_path = out_dir / f"{img_path.stem}__boxed{img_path.suffix}"
        cv2.imwrite(str(out_path), canvas)
        drawn += 1

    return {"drawn": drawn, "skipped": skipped}


In [ ]:
frames_dir = "../data/frames_out"
out_dir    = "../data/frames_out_annotated"

df = pd.read_csv("../data/biigle_files/26577-kok-20240219-buv-kok-060-01.csv")

summary = overlay_boxes_on_frames(df, frames_dir, out_dir)
summary

In [ ]:
from sftk.biigle_handler import BiigleHandler

bh = BiigleHandler()
exported_annotations = bh.read_csvs_from_zip_bytes(zip_bytes)
exported_annotations.keys()

In [ ]:
a = 0
for key in exported_annotations.keys():
    print(len(exported_annotations[key]))
    a+=len(exported_annotations[key])
print(a)

In [ ]:
type(exported_annotations)
a = bh.concat_csv_dict(exported_annotations)
a.shape

## Annotations per project level

In [ ]:
exported_raw_annotations = biigle_parser.process_video_annotations(PROJECT_ID, resource="projects", export_raw=True)


## Label per project level

videos are Done/Nothing here etcetc

In [ ]:
exported_raw_labels = biigle_parser.process_video_annotations(PROJECT_ID, resource="projects", export_raw=True, type_id=10)

#    "id" => 10,
#    "name" => "VideoLabels\Csv",

In [ ]:
exported_raw_labels["raw_annotations_df"] 

In [ ]:
erl = exported_raw_labels["raw_annotations_df"]
# erl[erl["label_name"] == ["Done"]]
# erl[(erl["label_name"] == "Done") | (erl["label_name"] == "Nothing here")]

In [ ]:
erl["label_name"].unique()

# Process = Can't annotate/ random / Interesitng sighting / to review
# - weekly

In [ ]:
done_erl = erl[(erl["label_name"] == "Done") | (erl["label_name"] == "Nothing here")].copy()
not_done_erl = erl[(erl["label_name"] == "In progress") | (erl["label_name"] == "To review")].copy()


not_done_erl.shape

In [ ]:
done_erl["base_video"] = done_erl["filename"].str.extract(r"^(.*?\.mp4)")
video_counts = done_erl["base_video"].value_counts()

done_erl["count_for_video"] = done_erl["base_video"].map(video_counts)

done_erl


In [ ]:
not_done_erl

not_done_erl["base_video"] = not_done_erl["filename"].str.extract(r"^(.*?\.mp4)")
video_counts = not_done_erl["base_video"].value_counts()

not_done_erl["count_for_video"] = not_done_erl["base_video"].map(video_counts)

not_done_erl["count_for_video"].unique()

In [ ]:
done_erl["count_for_video"].unique()

In [ ]:
exported_raw_annotations.keys(), exported_raw_labels.keys()

In [ ]:
era = exported_raw_annotations["raw_annotations_df"]
era[era["label_name"] == "Done"]

In [ ]:
exported_processed_project_annotations = biigle_parser.process_video_annotations(PROJECT_ID, resource="projects")


In [ ]:
type(exported_processed_project_annotations)
exported_processed_project_annotations.keys()


In [ ]:
exported_processed_project_annotations["max_n_df"][exported_processed_project_annotations["max_n_df"]["DropID"] == "TON_20211026_BUV_TON_016_01"]

exported_processed_project_annotations["max_n_df"]

In [ ]:
exported_processed_project_annotations["max_n_30s_df"]

In [ ]:
len(exported_processed_project_annotations["max_n_30s_df"]["video_filename"].unique())

In [ ]:
len(exported_processed_project_annotations["max_n_df"]["video_filename"].unique())

In [ ]:
# If all SurveyIDs had all 60 videos
27*60

In [ ]:
processed_annotations.keys()

In [ ]:
processed_annotations["raw_annotations_df"]

In [ ]:
# END